## 5. Batch Processing and Data Warehouse via Airflow & PySpark
5.1 Medallion Architecture Overview
Bronze Layer: Raw data extracted from PostgreSQL for each table.
Silver Layer: Data cleaning and transformation using PySpark.
Gold Layer: Aggregated data (e.g., average fares by pickup location) in a star schema format.
5.2 Airflow DAG Example
Below is an example DAG (save as nyc_taxi_dag.py in your Airflow DAGs folder):

In [1]:
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from datetime import datetime, timedelta
import pandas as pd
import psycopg2
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp, col, when, count, avg, expr

# ✅ Airflow DAG Configuration
default_args = {
    "owner": "airflow",
    "start_date": datetime(2024, 7, 1),
    "retries": 2,
    "retry_delay": timedelta(minutes=5),
}

dag = DAG("nyc_taxi_batch_pipeline", default_args=default_args, schedule_interval="@daily")

# ✅ Database Connection Details
DB_CONFIG = {
    "host": "localhost",
    "port": 5432,
    "database": "nyc_taxi",
    "user": "your_username",
    "password": "your_password"
}

TABLES = ["yellow_tripdata_2024_01", "yellow_tripdata_2024_02", "green_tripdata_2024_01", "green_tripdata_2024_02"]

# ✅ Extract Data and Save as Partitioned CSV
def extract_data(**kwargs):
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        os.makedirs("data/bronze", exist_ok=True)

        for table in TABLES:
            query = f"SELECT * FROM public.{table};"
            df = pd.read_sql(query, conn)

            # Save partitioned by taxi type
            taxi_type = "yellow" if "yellow" in table else "green"
            month = table[-2:]
            df.to_csv(f"data/bronze/{taxi_type}_tripdata_2024_{month}.csv", index=False)

        conn.close()
    except Exception as e:
        print(f"❌ Error extracting data: {str(e)}")
        raise

# ✅ Transform Data and Apply Feature Engineering in Spark
def transform_data(**kwargs):
    spark = SparkSession.builder.appName("NYC_Taxi_Transformation").getOrCreate()
    
    bronze_path = "data/bronze/"
    silver_path = "data/silver/"
    gold_path = "data/gold/"
    os.makedirs(silver_path, exist_ok=True)
    os.makedirs(gold_path, exist_ok=True)

    for file in os.listdir(bronze_path):
        if file.endswith(".csv"):
            df = spark.read.csv(bronze_path + file, header=True, inferSchema=True)

            # ✅ Convert timestamps and clean missing values
            df = df.withColumn("pickup_datetime", to_timestamp(col("pickup_datetime"), "yyyy-MM-dd HH:mm:ss"))\
                   .na.fill({"fare_amount": 0, "extra": 0, "mta_tax": 0, "tip_amount": 0, "tolls_amount": 0, 
                             "improvement_surcharge": 0, "total_amount": 0})

            # ✅ Feature Engineering: Compute Tip & Toll Ratio
            df = df.withColumn("tip_ratio", when(col("total_amount") > 0, col("tip_amount") / col("total_amount")).otherwise(0.0))\
                   .withColumn("toll_ratio", when(col("total_amount") > 0, col("tolls_amount") / col("total_amount")).otherwise(0.0))

            # ✅ Save Silver Layer (processed data)
            df.write.mode("overwrite").parquet(silver_path + file.replace(".csv", "_silver.parquet"))

            # ✅ Gold Layer Aggregations
            gold_df = df.groupBy("pulocationid")\
                        .agg(
                            count("*").alias("total_trips"),
                            avg("fare_amount").alias("avg_fare"),
                            expr("percentile_approx(fare_amount, 0.5)").alias("median_fare"),
                            expr("max(fare_amount)").alias("max_fare")
                        )

            gold_df.write.mode("overwrite").parquet(gold_path + file.replace(".csv", "_gold.parquet"))

    spark.stop()

# ✅ Define Airflow Tasks
t1 = PythonOperator(task_id="extract_data", python_callable=extract_data, dag=dag)
t2 = PythonOperator(task_id="transform_data", python_callable=transform_data, dag=dag)

# ✅ Set Task Dependencies
t1 >> t2


/var/folders/4n/68l65j717mxb8994g_2bc9_00000gn/T/ipykernel_75250/3608515548.py:2 DeprecationWarning: The `airflow.operators.python_operator.PythonOperator` class is deprecated. Please use `'airflow.operators.python.PythonOperator'`.

/var/folders/4n/68l65j717mxb8994g_2bc9_00000gn/T/ipykernel_75250/3608515548.py:18 RemovedInAirflow3Warning: Param `schedule_interval` is deprecated and will be removed in a future release. Please use `schedule` instead.

<Task(PythonOperator): transform_data>

## 6. Interactive Dashboard with Plotly Dash
Use Dash to create a dashboard that periodically reads prediction data from Redis. This example works for predictions from any table since each Redis key encodes the source table.

In [2]:
import dash
from dash import html, dcc
from dash.dependencies import Output, Input
import redis
import json
import plotly.express as px
import pandas as pd

r = redis.StrictRedis(host='localhost', port=6379, decode_responses=True)

def get_predictions():
    keys = r.keys("trip:*")  # Ensure correct key format
    data = []
    for key in keys:
        record = json.loads(r.get(key))
        record["key"] = key
        record["predicted_fare"] = record.get("fare_prediction", 0)  # ✅ Ensure this exists
        record["predicted_duration"] = record.get("duration_prediction", 0)  # ✅ Include duration prediction
        data.append(record)
    return data

app = dash.Dash(__name__)
app.layout = html.Div([
    html.H3("NYC Taxi Predictions Dashboard"),
    dcc.Interval(id='interval-component', interval=10*1000, n_intervals=0),
    dcc.Graph(id='fare-prediction-graph'),
    dcc.Graph(id='duration-prediction-graph')
])

@app.callback(
    [Output('fare-prediction-graph', 'figure'),
     Output('duration-prediction-graph', 'figure')],
    [Input('interval-component', 'n_intervals')]
)
def update_graph(n):
    data = get_predictions()
    if data:
        df = pd.DataFrame(data)
        
        # 🚖 Plot Fare Predictions
        fare_fig = px.bar(df, x="key", y="predicted_fare", title="Predicted Fare per Trip")

        # ⏳ Plot Duration Predictions
        duration_fig = px.bar(df, x="key", y="predicted_duration", title="Predicted Trip Duration per Trip")

    else:
        fare_fig = px.bar(title="No fare predictions available yet")
        duration_fig = px.bar(title="No duration predictions available yet")

    return fare_fig, duration_fig

if __name__ == '__main__':
    app.run_server(debug=True)


## 8. Summary
This complete solution now:

Ingests data from four PostgreSQL tables (yellow and green for two months each).
Simulates real‑time streaming by publishing rows (with a table identifier) to Kafka.
Processes streaming data with Spark Structured Streaming, calls a REST API to get fare predictions, and stores them in Redis.
Trains and deploys an XGBoost model served via Flask.
Orchestrates batch processing using Airflow with a medallion architecture.
Displays results interactively using a Plotly Dash dashboard.
All components run with free, open‑source tools and can be developed and executed within a Jupyter Notebook environment. Adjust the code snippets as needed to fit your data schema and deployment environment.